## Introduction

The problem addressed in this project is the prediction of property sale prices in the UK based on historical transaction data and property characteristics. Property price estimation is complex due to the number of factors that influence value, including property type, location, tenure, and timing of sale. This project applies machine learning techniques to identify patterns within historical data and use them to generate price predictions for new, unseen properties.

This assessment focuses on the development of a machine learning regression model that is trained, tested, and evaluated using secondary data. The aim of the project is to predict UK property prices based on a range of property and related attributes, and to demonstrate the complete machine learning workflow from raw data to deployment in a web application.

The dataset used in this project is sourced from the UK Land Registry Price Paid Data, which is an open and publicly available dataset containing records of residential property transactions across England and Wales. The dataset includes information such as sale price, date of transfer, property type, tenure, location, and whether a property is newly built.

Because the dataset contains known property prices, the problem is classified as supervised learning. As the target variable is a continuous numerical value, the task is treated as a regression problem. Linear regression models are therefore appropriate, and several regression approaches are tested to compare their effectiveness.

Before training the model, data preparation is required. The raw data is provided across two CSV files and includes a mixture of numerical, categorical, and administrative fields. These files are combined into a single dataset, after which data cleaning is performed. This includes handling missing values, removing duplicate records, filtering invalid transactions, converting data types, and removing irrelevant features. Feature engineering is also applied, such as transforming property prices using a logarithmic scale and extracting year and month information from transaction dates.

Once the data is prepared, several regression models are trained and evaluated, including baseline linear regression, Ridge regression, and Lasso-style regularisation. Model performance is assessed using standard regression metrics such as Variance (R^2), Mean Absolute Error (MAE), and Root Mean Squared Error (RMSE).

Finally, the Ridge regression model, which produced the most accurate predictions based on evaluation metrics, is saved and integrated into a flask web application. This application allows users to enter property details and receive an estimated UK property price.

In [22]:
import pandas as pd
import numpy as np

## Data Loading

The UK Land Registry Price Paid Data is provided across two CSV files. In this section, the files are loaded and combined into a single dataset to create a unified source for analysis and modelling.


In [23]:
# imports the two CSV files and concatenates them into a single DataFrame.
df1 = pd.read_csv("../data/pp-2023-part1.csv", header=None)
df2 = pd.read_csv("../data/pp-2023-part2.csv", header=None)
df = pd.concat([df1, df2], ignore_index=True)

df.shape # displays the number of rows and columns to show the size of the dataset

(856736, 16)

In [24]:
df.head() # displays the first few rows of the dataset to provide an initial overview of the data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,{0E082196-CE18-5C09-E063-4704A8C0A10E},221000,2023-09-22 00:00,PL6 6JX,T,N,F,3,NaN,PILLAR WALK,NaN,PLYMOUTH,CITY OF PLYMOUTH,CITY OF PLYMOUTH,A,A
1,{0E082196-CE19-5C09-E063-4704A8C0A10E},228000,2023-08-25 00:00,PL7 1SJ,S,N,F,102,NaN,MERAFIELD ROAD,NaN,PLYMOUTH,CITY OF PLYMOUTH,CITY OF PLYMOUTH,A,A
2,{0E082196-CE1A-5C09-E063-4704A8C0A10E},480000,2023-10-26 00:00,TQ6 0AS,F,N,L,1A,NaN,RIVER VIEW,KINGSWEAR,DARTMOUTH,SOUTH HAMS,DEVON,A,A
3,{0E082196-CE1B-5C09-E063-4704A8C0A10E},625000,2023-07-14 00:00,TQ1 2HB,D,N,F,14,NaN,OXLEA CLOSE,NaN,TORQUAY,TORBAY,TORBAY,A,A
4,{0E082196-CE1C-5C09-E063-4704A8C0A10E},174000,2023-08-04 00:00,PL2 1LL,T,N,F,58,NaN,ST AUBYN AVENUE,NaN,PLYMOUTH,CITY OF PLYMOUTH,CITY OF PLYMOUTH,A,A


## Assigning Column Names

The raw dataset does not include column headers. Official UK Land Registry column names are assigned to improve readability and allow meaningful analysis and feature selection.


In [25]:
# assigns column names.
df.columns = [
    "transaction_id",
    "price",
    "date_of_transfer",
    "postcode",
    "property_type",
    "new_build",
    "tenure",
    "paon",
    "saon",
    "street",
    "locality",
    "town_city",
    "district",
    "county",
    "ppd_category_type",
    "record_status"
]

df.head()


,transaction_id,price,date_of_transfer,postcode,property_type,new_build,tenure,paon,saon,street,locality,town_city,district,county,ppd_category_type,record_status
0,{0E082196-CE18-5C09-E063-4704A8C0A10E},221000,2023-09-22 00:00,PL6 6JX,T,N,F,3,NaN,PILLAR WALK,NaN,PLYMOUTH,CITY OF PLYMOUTH,CITY OF PLYMOUTH,A,A
1,{0E082196-CE19-5C09-E063-4704A8C0A10E},228000,2023-08-25 00:00,PL7 1SJ,S,N,F,102,NaN,MERAFIELD ROAD,NaN,PLYMOUTH,CITY OF PLYMOUTH,CITY OF PLYMOUTH,A,A
2,{0E082196-CE1A-5C09-E063-4704A8C0A10E},480000,2023-10-26 00:00,TQ6 0AS,F,N,L,1A,NaN,RIVER VIEW,KINGSWEAR,DARTMOUTH,SOUTH HAMS,DEVON,A,A
3,{0E082196-CE1B-5C09-E063-4704A8C0A10E},625000,2023-07-14 00:00,TQ1 2HB,D,N,F,14,NaN,OXLEA CLOSE,NaN,TORQUAY,TORBAY,TORBAY,A,A
4,{0E082196-CE1C-5C09-E063-4704A8C0A10E},174000,2023-08-04 00:00,PL2 1LL,T,N,F,58,NaN,ST AUBYN AVENUE,NaN,PLYMOUTH,CITY OF PLYMOUTH,CITY OF PLYMOUTH,A,A


In [26]:
df.info() # shows information about the dataset, including column names, data types, and non-null counts


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856736 entries, 0 to 856735
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   transaction_id     856736 non-null  object
 1   price              856736 non-null  int64 
 2   date_of_transfer   856736 non-null  object
 3   postcode           854495 non-null  object
 4   property_type      856736 non-null  object
 5   new_build          856736 non-null  object
 6   tenure             856736 non-null  object
 7   paon               856736 non-null  object
 8   saon               118592 non-null  object
 9   street             842512 non-null  object
 10  locality           322872 non-null  object
 11  town_city          856736 non-null  object
 12  district           856736 non-null  object
 13  county             856736 non-null  object
 14  ppd_category_type  856736 non-null  object
 15  record_status      856736 non-null  object
dtypes: int64(1), object(

## Data Cleaning and Filtering

This section focuses on cleaning the raw dataset. Duplicate records are removed, invalid transactions are filtered out, and administrative metadata that is not useful for prediction is discarded.

In [27]:
df.isnull().sum() # identifies the number of missing values in each column


transaction_id            0
price                     0
date_of_transfer          0
postcode               2241
property_type             0
new_build                 0
tenure                    0
paon                      0
saon                 738144
street                14224
locality             533864
town_city                 0
district                  0
county                    0
ppd_category_type         0
record_status             0
dtype: int64

In [28]:
df.describe(include="all") # generates summary statistics for all columns to understand data distributions and data quality


,transaction_id,price,date_of_transfer,postcode,property_type,new_build,tenure,paon,saon,street,locality,town_city,district,county,ppd_category_type,record_status
count,856736,8.567360e+05,856736,854495,856736,856736,856736,856736,118592,842512,322872,856736,856736,856736,856736,856736
unique,856736,NaN,365,489858,5,2,2,73857,8670,171810,16068,1146,335,114,2,1
top,{0E082196-CE18-5C09-E063-4704A8C0A10E},NaN,2023-06-30 00:00,L6 1AT,T,N,F,1,FLAT 1,HIGH STREET,SHIRLEY,LONDON,BIRMINGHAM,GREATER LONDON,A,A
freq,1,NaN,15575,147,230963,760338,648788,22318,5531,5884,944,59301,11661,95760,705629,856736
mean,NaN,4.055127e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,1.523214e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,1.750000e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,2.750000e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,4.250000e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# removes duplicate rows from the dataset.
df = df.drop_duplicates()
df.duplicated().sum()


np.int64(0)

In [30]:
# keeps only standard residential transactions and valid records.
df = df[df["ppd_category_type"] == "A"]
df = df[df["record_status"] == "A"]

# Removes administrative metadata columns.
df = df.drop(columns=["ppd_category_type", "record_status"])

df.shape


(705629, 14)

## Data Type Conversion and Handling Missing Values

Key variables such as price and date of transfer are converted into appropriate numerical formats. Rows with missing values in essential fields are removed to ensure the dataset is suitable for machine learning.


In [31]:
# converts price to numeric and date_of_transfer to datetime for analysis and feature extraction.
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["date_of_transfer"] = pd.to_datetime(df["date_of_transfer"], errors="coerce")

df[["price", "date_of_transfer"]].head()


,price,date_of_transfer
0,221000,2023-09-22
1,228000,2023-08-25
2,480000,2023-10-26
3,625000,2023-07-14
4,174000,2023-08-04


In [32]:
# drops rows with missing key fields required for modelling.
df = df.dropna(subset=["price", "date_of_transfer", "property_type", "new_build", "tenure", "county"])
df.isnull().sum()


transaction_id           0
price                    0
date_of_transfer         0
postcode                91
property_type            0
new_build                0
tenure                   0
paon                     0
saon                614378
street                9787
locality            432399
town_city                0
district                 0
county                   0
dtype: int64

## Feature Selection and Transformation

Irrelevant identifiers and address-level fields are removed to reduce noise. A logarithmic transformation is applied to property prices to reduce the impact of extreme values and improve model performance.

In [33]:
# removes identifiers and address fields that do not contribute meaningfully to prediction.
df = df.drop(columns=[
    "transaction_id",
    "postcode",
    "paon",
    "saon",
    "street",
    "locality",
    "town_city",
    "district"
], errors="ignore")

df.columns.tolist()


['price', 'date_of_transfer', 'property_type', 'new_build', 'tenure', 'county']

In [34]:
# uses a log transformation to reduce the impact of extreme property prices.
df["log_price"] = np.log1p(df["price"])
df[["price", "log_price"]].head()


,price,log_price
0,221000,12.305923
1,228000,12.337105
2,480000,13.081543
3,625000,13.345509
4,174000,12.066816


## Feature Engineering

Additional features are created by extracting the year and month from the transaction date. These features help capture changes in the housing market over time.

In [35]:
# extracts year and month to capture market trends over time.
df["year"] = df["date_of_transfer"].dt.year
df["month"] = df["date_of_transfer"].dt.month

# drops original date column after extracting features.
df = df.drop(columns=["date_of_transfer"])

df[["year", "month"]].head()


,year,month
0,2023,9
1,2023,8
2,2023,10
3,2023,7
4,2023,8


In [36]:
# defines the input features the web app will collect and the target variable.
feature_cols = ["property_type", "new_build", "tenure", "county", "year", "month"]
X = df[feature_cols].copy()
y = df["log_price"].copy()

X.head()


,property_type,new_build,tenure,county,year,month
0,T,N,F,CITY OF PLYMOUTH,2023,9
1,S,N,F,CITY OF PLYMOUTH,2023,8
2,F,N,L,DEVON,2023,10
3,D,N,F,TORBAY,2023,7
4,T,N,F,CITY OF PLYMOUTH,2023,8


## Train and Test Split

The dataset is split into training and testing sets to allow the model to be trained on historical data and evaluated on unseen data.

In [37]:
from sklearn.model_selection import train_test_split

# splits the dataset into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


## Baseline Model and Preprocessing

A baseline linear regression model is implemented to establish an initial performance reference. Preprocessing is applied using a ColumnTransformer to handle categorical and numerical features appropriately.


In [38]:
from sklearn.compose import ColumnTransformer # lets us apply different preprocessing steps to different columns
from sklearn.preprocessing import OneHotEncoder, StandardScaler # OneHotEncoder turns categories into numeric columns and StandardScaler scales numeric features
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression # regression model that learns a linear relationship between inputs and the target

categorical_features = ["property_type", "new_build", "tenure", "county"] # categories
numeric_features = ["year", "month"]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", StandardScaler(), numeric_features)
    ]
)

# creates ML pipeline that preprocesses the raw input features and trains a linear regression model on the processed data
baseline_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LinearRegression())
])
# trains the pipeline on the training data
baseline_model.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...), ('num', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

## Regularised Regression Models

To improve performance and address limitations of the baseline model, Ridge regression and Lasso-style regularisation are applied. These approaches help control model complexity when working with a large number of encoded features.


In [39]:
from sklearn.linear_model import Ridge, SGDRegressor # Ridge and Lasso-style regression models with regularisation

# Ridge regression pipeline
ridge_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", Ridge(alpha=1.0))
])

# Lasso regression pipeline
lasso_model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", SGDRegressor(
        penalty="l1",
        alpha=0.0001,
        max_iter=2000,
        tol=1e-3,
        random_state=42
    ))
])

ridge_model.fit(X_train, y_train) # trains the Ridge regression model on the training data
lasso_model.fit(X_train, y_train) # trains the Lasso regression model on the training data


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...), ('num', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

## Model Evaluation

The trained models are evaluated using standard regression metrics to assess accuracy and reliability. Results from different models are compared to identify the most suitable approach.

In [40]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score # imports evaluation metrics used to measure regression model performance

def evaluate(model, X_test, y_test, name="Model"):
    preds = model.predict(X_test) # generates predictions using the trained model
    mae = mean_absolute_error(y_test, preds) # calculates mean absolute error
    rmse = np.sqrt(mean_squared_error(y_test, preds)) # calculates root mean squared error
    r2 = r2_score(y_test, preds) # calculates how much variance in the data is explained by the model

    # prints evaluation results
    print(f"{name} results:")
    print(f"  R^2   = {r2:.4f}")
    print(f"  MAE  = {mae:.4f}")
    print(f"  RMSE = {rmse:.4f}")
    return r2, mae, rmse

evaluate(baseline_model, X_test, y_test, "Baseline Linear Regression") # evaluates the baseline linear regression model
evaluate(ridge_model, X_test, y_test, "Ridge Regression") # evaluates the Ridge regression model
evaluate(lasso_model, X_test, y_test, "Lasso (SGD L1)") # evaluates the Lasso regression model


Baseline Linear Regression results:
  R^2   = 0.5872
  MAE  = 0.3097
  RMSE = 0.4199
Ridge Regression results:
  R^2   = 0.5872
  MAE  = 0.3097
  RMSE = 0.4199
Lasso (SGD L1) results:
  R^2   = 0.5857
  MAE  = 0.3102
  RMSE = 0.4207


(0.5857289719252332, 0.3102427737416894, np.float64(0.42070144385578445))

## Model Saving

The best-performing model is saved using joblib so it can be reused later and integrated into a web application without retraining.


In [41]:
import joblib # used to save and load trained machine learning models
joblib.dump(ridge_model, "../model/uk_house_price_model.joblib") # saves the trained regression model
print("Saved")

Saved
